<a href="https://colab.research.google.com/github/aquafire088/invoice-processor/blob/main/Snippets_Importing_libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio spaces transformers accelerate numpy requests torch torchvision qwen-vl-utils av ipython reportlab fpdf python-docx pillow huggingface_hub

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.6 MB/s eta 0:00:00
   ━━━━

In [1]:
import torch
import gradio as gr
import os
from threading import Thread
from PIL import Image
from transformers import AutoProcessor, TextIteratorStreamer
from huggingface_hub import login
import traceback
from getpass import getpass

In [2]:
# Import sécurisé pour le modèle Qwen2-VL
try:
    from transformers.models.qwen2_vl import Qwen2VLForConditionalGeneration
    print("✅ Qwen2VLForConditionalGeneration importé avec succès")
except ImportError:
    try:
        from transformers import AutoModelForVision2Seq as Qwen2VLForConditionalGeneration
        print("⚠️  Utilisation d'AutoModelForVision2Seq comme fallback")
    except ImportError:
        print("❌ Impossible d'importer le modèle Qwen2-VL")
        raise ImportError("Modèle Qwen2-VL non supporté dans cette version de transformers")

✅ Qwen2VLForConditionalGeneration importé avec succès


In [3]:
# Define model options
MODEL_ID = {
    "OCR-KIE": "prithivMLmods/Qwen2-VL-OCR-2B-Instruct",
}

# Preload models and processors into CUDA
models = {}
processors = {}
for name, model_id in MODEL_ID.items():
    print(f"Loading {name}...")
    model = Qwen2VLForConditionalGeneration.from_pretrained(
        model_id,
        trust_remote_code=True,
        torch_dtype=torch.float16
    ).to("cuda").eval()
    processors[name] = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

image_extensions = Image.registered_extensions()

Loading OCR-KIE...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/4.42G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/408 [00:00<?, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [12]:
# Configuration du modèle
MODEL_ID = "prithivMLmods/Qwen2-VL-OCR-2B-Instruct"

# Authentification Hugging Face sécurisée
def setup_huggingface_auth():
    """Configure l'authentification Hugging Face de manière sécurisée"""
    try:
        # Priorité aux variables d'environnement
        hf_token = os.getenv('HF_TOKEN') or os.getenv('HUGGINGFACE_TOKEN')

        if not hf_token:
            print("⚠️  Token HF non trouvé dans les variables d'environnement")
            # En mode interactif, demander le token
            if hasattr(__builtins__, '__IPYTHON__'):
                hf_token = getpass("Entrez votre token Hugging Face: ")
            else:
                print("Mode non-interactif: définissez HF_TOKEN dans vos variables d'environnement")
                return False

        if hf_token:
            login(hf_token)
            print("✅ Authentification Hugging Face réussie")
            return True
    except Exception as e:
        print(f"⚠️  Échec de l'authentification HF: {str(e)}")
        return False

In [4]:
# Configuration du device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  Device utilisé: {device}")

# Initialisation des variables globales
MODEL_LOADED = False
model = None
processor = None

def load_model():
    """Charge le modèle et le processeur"""
    global model, processor, MODEL_LOADED

    print("🔄 Chargement du modèle en cours...")
    try:
        # Authentification
        auth_success = setup_huggingface_auth()
        if not auth_success:
            print("⚠️  Tentative de chargement sans authentification")

        # Chargement du modèle
        model = Qwen2VLForConditionalGeneration.from_pretrained(
            MODEL_ID,
            trust_remote_code=True,
            torch_dtype=torch.float16,
            low_cpu_mem_usage=False
        ).eval()

        # Placement manuel sur CPU si nécessaire
        if device == "cpu":
            model = model.to(device)

        # Chargement du processeur
        processor = AutoProcessor.from_pretrained(
            MODEL_ID,
            trust_remote_code=True
        )

        image_extensions = Image.registered_extensions()

        print("✅ Modèle chargé avec succès!")
        MODEL_LOADED = True
        return True

    except Exception as e:
        print(f"❌ Erreur lors du chargement du modèle: {str(e)}")
        traceback.print_exc()
        MODEL_LOADED = False
        model = None
        processor = None
        return False

🖥️  Device utilisé: cuda


In [5]:
def diagnose_model():
    """Diagnostic du modèle et de l'environnement"""
    print("\n🔍 DIAGNOSTIC DU SYSTÈME")
    print("=" * 50)

    # Vérification PyTorch
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA disponible: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
        print(f"Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

    # Vérification du modèle
    print(f"Modèle chargé: {MODEL_LOADED}")
    if MODEL_LOADED:
        print(f"Device du modèle: {model.device}")
        print(f"Type du modèle: {type(model)}")
        print(f"Processeur: {type(processor)}")

    # Test basique
    if MODEL_LOADED:
        try:
            # Test du tokenizer
            test_text = "Test"
            tokens = processor.tokenizer(test_text, return_tensors="pt")
            print(f"✅ Tokenizer fonctionne: {tokens['input_ids'].shape}")
        except Exception as e:
            print(f"❌ Erreur tokenizer: {e}")

    print("=" * 50)

In [6]:
# Fonction de test simplifiée
def test_model_simple():
    """Test basique du modèle"""
    if not MODEL_LOADED:
        return "❌ Modèle non chargé"

    try:
        # Création d'une image test
        test_image = Image.new('RGB', (100, 100), color='white')
        test_text = "Décrivez cette image"

        # Test des inputs
        inputs = processor(
            text=test_text,
            images=[test_image],
            return_tensors="pt"
        )

        # Test de génération minimal
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,
                do_sample=False
            )

        result = processor.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return f"✅ Test réussi: {result[:100]}..."

    except Exception as e:
        return f"❌ Test échoué: {str(e)}"

In [7]:
# Extensions d'images supportées
SUPPORTED_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp')

def format_plain_text(output_text):
    """Nettoie le texte de sortie"""
    if not output_text:
        return ""
    return output_text.replace("<|im_end|>", "").strip()

In [8]:
def generate_strict_invoice_prompt(selected_fields):
    """Génère un prompt strict pour l'extraction de factures"""
    if not selected_fields:
        return "⚠️ Veuillez sélectionner au moins un champ à extraire."

    field_mapping = {
        "Type de document": "type_document",
        "Numéro de facture": "numero_facture",
        "Référence client": "reference_client",
        "Date de facture": "date_facture",
        "Date d'échéance": "date_echeance",
        "Nom du client": "nom_client",
        "Client final": "client_final",
        "Adresse": "adresse_client",
        "ICE (client/fournisseur)": "ice_client",
        "Référence article": "reference_article",
        "Désignation": "designation",
        "Quantité": "quantite",
        "P.U. Brut": "pu_brut",
        "Remise": "remise",
        "P.U. Net": "pu_net",
        "Montant HT": "montant_ht",
        "TVA": "montant_tva",
        "Montant TTC": "montant_ttc",
        "Montant en lettres": "montant_lettres",
        "Mode de paiement": "mode_paiement",
        "RIB": "rib",
        "Adresse société": "adresse_societe",
        "R.C.": "rc",
        "C.N.S.S": "cnss",
        "Patente": "patente",
        "I.F": "if"
    }

    valid_fields = [f for f in selected_fields if f in field_mapping]
    if not valid_fields:
        return "❌ Aucun champ valide sélectionné."

    json_structure = "{\n" + ",\n".join([f'  "{field_mapping[f]}": ""' for f in valid_fields]) + "\n}"

    prompt = (
        "**Extraction de facture**\n\n"
        "Analysez précisément l'image de facture et extrayez les champs demandés.\n\n"
        "**Champs requis**:\n" + "\n".join(f"- {f}" for f in valid_fields) +
        "\n\n**Format de sortie**:\n```json\n" + json_structure + "\n```\n\n"
        "**Instructions**:\n"
        "- Retournez uniquement le JSON\n"
        "- Utilisez les valeurs exactes de la facture\n"
        "- Champs manquants = \"\"\n"
        "- Pas de traduction\n"
        "- Formatage numérique exact (ne pas modifier virgules/points)"
    )
    return prompt

def generate_prompt_from_fields(general, client, items, totals, payment, company):
    """Combine les champs sélectionnés"""
    all_fields = general + client + items + totals + payment + company
    return generate_strict_invoice_prompt(all_fields)

In [9]:
def qwen_inference(media_input, text_input):
    """Simple Qwen2-VL inference function"""

    # Basic validation
    if not MODEL_LOADED or model is None or processor is None:
        yield "❌ Model not loaded"
        return

    if not text_input or not text_input.strip():
        yield "❌ No prompt provided"
        return

    if media_input is None:
        yield "❌ No image provided"
        return

    try:
        # Load image
        if isinstance(media_input, str):
            image = Image.open(media_input)
        else:
            image = media_input

        # Convert to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')

        yield "🔄 Processing..."

        # Process inputs
        inputs = processor(
            text=text_input,
            images=[image],
            return_tensors="pt"
        )

        # Move to device
        inputs = {k: v.to(model.device) if hasattr(v, 'to') else v
                 for k, v in inputs.items()}

        # Generate
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=512,
                temperature=0.1,
                do_sample=False
            )

        # Decode result
        generated_text = processor.tokenizer.decode(
            outputs[0][inputs["input_ids"].shape[1]:],
            skip_special_tokens=True
        )

        yield generated_text if generated_text else "⚠️ No result generated"

    except torch.cuda.OutOfMemoryError:
        yield "❌ GPU memory error - try smaller image"
    except Exception as e:
        yield f"❌ Error: {str(e)}"

In [10]:
# Interface Gradio
def create_interface():
    """Crée l'interface Gradio"""
    with gr.Blocks(
        title="Extracteur de Factures Qwen2",
        css="""
        .gradio-container {
            max-width: 1200px !important;
        }
        .gr-button-primary {
            background: linear-gradient(45deg, #2196F3, #21CBF3) !important;
        }
        """
    ) as demo:
        gr.Markdown("## 📄 Extraction de Données de Factures - Qwen2-VL")

        # Statut du modèle
        status_msg = "✅ Modèle chargé avec succès" if MODEL_LOADED else "❌ Modèle non chargé"
        gr.Markdown(f"**Statut**: {status_msg}")

        with gr.Row():
            with gr.Column(scale=1):
                gr.Markdown("### 1. Sélection des champs")
                with gr.Group():
                    general = gr.CheckboxGroup(
                        ["Type de document", "Numéro de facture", "Référence client",
                         "Date de facture", "Date d'échéance"],
                        label="📋 Informations Générales"
                    )
                    client = gr.CheckboxGroup(
                        ["Nom du client", "Client final", "Adresse", "ICE (client/fournisseur)"],
                        label="👤 Client"
                    )
                    items = gr.CheckboxGroup(
                        ["Référence article", "Désignation", "Quantité", "P.U. Brut",
                         "Remise", "P.U. Net", "Montant HT"],
                        label="🛍️ Ligne d'articles"
                    )
                    totals = gr.CheckboxGroup(
                        ["Montant HT", "TVA", "Montant TTC", "Montant en lettres"],
                        label="💰 Totaux"
                    )
                    payment = gr.CheckboxGroup(
                        ["Mode de paiement", "RIB"],
                        label="💳 Paiement"
                    )
                    company = gr.CheckboxGroup(
                        ["Adresse société", "R.C.", "C.N.S.S", "Patente", "I.F"],
                        label="🏢 Coordonnées société"
                    )

                generate_btn = gr.Button("🔧 Générer le prompt", variant="primary")

            with gr.Column(scale=1):
                gr.Markdown("### 2. Configuration de l'extraction")
                prompt_box = gr.Textbox(
                    label="📝 Prompt généré",
                    lines=8,
                    placeholder="Le prompt apparaîtra ici...",
                    show_copy_button=True
                )
                image_input = gr.Image(
                    label="📤 Téléverser une facture",
                    type="filepath",
                    sources=["upload"]
                )
                extract_btn = gr.Button(
                    "🚀 Lancer l'extraction",
                    variant="primary",
                    size="lg"
                )
                output_textbox = gr.Textbox(
                    label="📊 Résultats d'extraction",
                    lines=10,
                    interactive=False,
                    show_copy_button=True
                )

        # Gestion des événements
        generate_btn.click(
            generate_prompt_from_fields,
            inputs=[general, client, items, totals, payment, company],
            outputs=prompt_box
        )

        extract_btn.click(
            qwen_inference,
            inputs=[image_input, prompt_box],
            outputs=output_textbox
        )

    return demo

# Lancement de l'application
if __name__ == "__main__":
    demo = create_interface()
    demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://44b15c8a1c6dfa1334.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
